In [ ]:
import numpy as np
import scipy.stats as ss
from matplotlib import pyplot as plt

# Coding section 1
## Let's simulate Newcomb's astronomical observations

Let us suppose the \"good\" observations are normally distributed with mean $ 0 $ and standard deviation $ 10^{-1/2} $, and \"bad\" observations (from the contaminating distribution) are also normally distributed with mean $ 0 $ but with larger standard deviation $ 5\cdot10^{-1/2} $.  Let $ \epsilon=0.05 $, that is, there is a $ 95\% $ chance we draw a \"good\" observation and a $ 5\% $ chance we draw a contaminated observation.

In [ ]:
np.random.seed(123)
epsilon = 0.05 
data100 = []
#Let's take n=100 samples
for j in range(100):
    if np.random.binomial(1,epsilon):
        data100.append(np.random.normal(0,5*10**(-1/2)))
    else:
        data100.append(np.random.normal(0,10**(-1/2)))
plt.clf()
plt.hist(data100,density=True,bins=8)
x = np.linspace(min(data100),max(data100),100)
plt.plot(x,ss.norm.pdf(x,0,1/np.sqrt(10)),color='green',label='Idealized measurements')

print(r'p-value from Wilks-Shapiro test of normality with $n=100$: '+str(ss.shapiro(data100)[1]))
#With a sample size of 100 the contaminated data appear almost indistinguishable from truly Gaussian data

Hmmm... The data appear consistent with the hypothesis that $P$ is a Gaussian model.

In [ ]:
data500 = []
for j in range(500):
    if np.random.binomial(1,epsilon):
        data500.append(np.random.normal(0,5*10**(-1/2)))
    else:
        data500.append(np.random.normal(0,10**(-1/2)))
plt.clf()
plt.hist(data500,density=True,bins=10)
x = np.linspace(min(data500),max(data500),100)
plt.plot(x,ss.norm.pdf(x,0,1/np.sqrt(10)),color='green',label='Idealized measurements')
print(r'p-value from Wilks-Shapiro test of normality with $n=500$: '+str(ss.shapiro(data500)[1]))


With a larger sample data can be easily distinguished from Gaussian. How much deviation from the Gaussian assumption is acceptable? What sample size would be required to conclude that $P$ is in fact _not_ Gaussian if only $1\%$ of observations appear Gaussian?

# Coding section 2
## Testing the hypothesis that $P$ is independent

Given a $2$-dimensional binary source $P$ which has $P((0,0))=P((0,1))=0.1$, $P((1,0))=0.3$, and $P((1,1,))=0.5$, how much data do we need to collect before we can correctly identify that $P$ does not have independent marginals?

In [ ]:
np.random.seed(123)

P = np.array([[0.1,0.3],[0.1,0.5]])
#Let's try a sample of size n=100
data100 = np.random.multinomial(100,P.flatten())
print(r'p-value for the chi-squared test that $P$ is an independent model when $n=100$: \
'+str(ss.chi2_contingency(data100.reshape((2,2)))[1]))

Despite the fact that the space we are estimating probabilities in only has four outcomes, $100$ samples don't provide compelling enough evidence for us to say that $P$ is not independent!

In [ ]:
#Let's increase the sample size. Try n=500.
data500 = np.random.multinomial(500,P.flatten())
ss.chi2_contingency(data500.reshape((2,2)))
print(r'p-value for the chi-squared test that $P$ is an independent model when $n=500$: '\
+str(ss.chi2_contingency(data500.reshape((2,2)))[1]))

Huh, this is still not very strong evidence! Let's try collecting an enormous amount of data, $n=1000$.

In [ ]:
data1000 = np.random.multinomial(1000,P.flatten())
print(r'p-value for the chi-squared test that $P$ is an independent model when $n=1000$: '\
+str(ss.chi2_contingency(data1000.reshape((2,2)))[1]))

Finally we can say with some confidence that $X$ and $Y$ have some dependence. However, this p-value may not appear that significant after multiple-hypothesis-testing correction (e.g. if we are testing whether the appearance of a purine or pyrimidine in consecutive positions is independent across thousands of transcription factor binding sites).

# Coding section 3
## Computing the weight of the independent models in $P$

To begin, let's choose a selection of independent models $Q$ so that our set can approximate _any_ independent model. Each independent model can be described by two parameters, $q_x$ and $q_y$ in $[0,1]$, so we decide to place a grid over $q_x$ and $q_y$ and, for each pair of parameters, form the associated independent model (i.e. $Q((0,1))=q_x(1-q_y)$, etc.)

In [ ]:
#Let's put a 1% grid over the independent distributions on {0,1}^2
q_x = np.linspace(0,1,100)
q_y = np.linspace(0,1,100)
Lambda = 0.0 #This records the largest possible weight for an independent model seen so far
Qprime = np.zeros((2,2)) # This records the independent model achieving the largest weight in P
for i in range(100):
    for j in range(100):
        Q = np.array([[(1-q_x[i])*(1-q_y[j]),q_x[i]*(1-q_y[j])],[(1-q_x[i])*q_y[j],q_x[i]*q_y[j]]])
        #Create the independent source Q with marginals q_x and p_y for each combination of p_x and p_y
        if min((P/Q)[np.nonzero(Q)])>Lambda:
            Lambda = min((P/Q)[np.nonzero(Q)])
            #Compute the latent weight of the single model Q in P, take the largest
            Qprime = Q
print('Approximate latent weight of independent models in P: '+str(Lambda))
print('Approximate largest independent model in P: '+str(Qprime))

A surprisingly large amound of data from $P$ can be attributed to the independent model above!

# Coding section 4
## Examining independence in transcription factor binding sites from JASPAR

These data from JASPAR represent 594 ChIP-seq experiments, each for a different transcription factor. Each file has a characteristic $k$ (between $4$ and $26$), the length of the DNA $k$-mer bound by each transcription factor. Then the data from each experiment takes values in $\{A,C,G,T\}^k$. Each file has $k$ columns

In [ ]:
import binary_alphabet_independent_weight as iw #Contains functions to more explicitly find independent weight
import os


samples = os.listdir('./TFBSData/')
data = []
for j in range(len(samples)):
    f = open('./TFBSData/'+samples[j],'r')
    data.append(f.readlines())
    f.close()                      #Reading in k-mers for various transcription factors
    

For greater simplicity we don't look at $k$-mers of the $4$ DNA bases, but instead \"lump\" sources into $k$-mers of purines and pyrimidines (i.e. we look at $k$-dimensional joint sources with a binary alphabet). 

In [ ]:
purPyrBinData = []
for j in range(len(data)):
    purPyrBinData.append(iw.DNASeq2Phat(data[j])) #For each transcription factor, say data are length d strings
                                                    #gives the empirical distribution over {purine,pyrimidine}^d
    

short_targets = [x for x in purPyrBinData if 4<len(x)<=32] #Look only at 4 and 5-mers (computational efficiency)
short_target_indices = [i for i in range(len(purPyrBinData)) if 4<len(purPyrBinData[i])<=32]

for i in range(len(short_targets)):
    out = iw.numerical_ind_weight(short_targets[i])
    print('Independent weight of k-mers bound by '+samples[short_target_indices[i]][:-6]+': '+str(out[0][0]))
    s = ''
    for j in range(len(out[1])):
        s += 'q'+str(j+1)+'='+str(out[1][j])+',  '
    s = s[:-3]
    print('Optimal Bernoulli parameters for k-mers bound by '+samples[short_target_indices[i]][:-6]+':\n'+\
         s+'\n')

Some of these transcription factors truly appear to bind $k$-mers treating positions as independent in the probability of a purine or pyrimidine in each position.

# Coding section 5
## Finding the latent weight of the Poisson distributions in $P$

This algorithm works by acknowledging that when $\alpha\approx0$, $F(\alpha) = P(0)e^{\alpha}$. For each other $\frac{P(i)\cdot i!}{\alpha^i\cdot e^{-\alpha}}$ (and the \"tail\" function), we compute the unique $\alpha$ where it intersects with $P(0)e^\alpha$. If the function $\frac{P(i)\cdot i!}{\alpha^i\cdot e^{-\alpha}}$ intersects with $P(0)e^\alpha$ at the smallest $\alpha$ out of all functions, we know that $F(\alpha)=\frac{P(i)\cdot i!}{\alpha^i\cdot e^{-\alpha}}$ in an interval whose left-hand endpoint is that intersection $\alpha$.

In [ ]:
import poisson_weight as pois
#Bortkiewicz's horse kick data
horse_kick_deaths = np.array([144,91,32,11,2]) #Ignore 5+, which have no observations, i.e. L=4
mean = (0*144+1*91+2*32+3*11+4*2)/sum(horse_kick_deaths)

Let's use a chi-squared goodness-of-fit test to see whether the number of deaths by horse kick appears to have a Poisson distribution

In [ ]:
expected = sum(horse_kick_deaths)*np.exp(-mean)*np.array([mean**0,mean**1/np.math.factorial(1),mean**2/np.math.factorial(2),\
                                  mean**3/np.math.factorial(3),mean**4/np.math.factorial(4)])
print('Maximum likelihood estimate for the Poisson rate of P: '+str(mean))
print('p-value of the chi-squared G.O.F. test for Bortkiewicz\'s data having Poisson distribution: '\
      +str(ss.chisquare(horse_kick_deaths,expected)[1]))

Given this is a classical dataset long-believed to be an example of the Poisson law, it makes sense that it would pass a hypothesis test for Poisson-ness. But what weight do all the Poisson distributions achieve as a component of the source?

In [ ]:
P = horse_kick_deaths/sum(horse_kick_deaths)
print('Weight of Poisson distributions in P: '+str(pois.poisson_weight(P)[0]))
print('Rate of the largest Poisson component of P: '+str(pois.poisson_weight(P)[1]))

Indeed, almost $97\%$ of samples can be attributed to a Poisson model which should be expected from Bortkiewicz's source. However, this large fraction of samples originate from the Poisson$(0.63)$ distribution, whereas by maximum likelihood we would estimate the source to be a Poisson$(0.7)$ distribution...

What if the true source is exactly that of the empirical distribution from Bortkiewicz's data? That is, what if the estimated probabilities are the underlying true source, but we collect a much larger sample?

In [ ]:
resampled_horse_kicks = np.random.multinomial(5000,P) #Resampling the empirical distribution
resampled_P = resampled_horse_kicks/sum(resampled_horse_kicks)
resampled_mean = np.dot(resampled_horse_kicks,[0,1,2,3,4])/sum(resampled_horse_kicks)
resampled_expected = sum(resampled_horse_kicks)*np.exp(-resampled_mean)*np.array([1,resampled_mean,\
                                            resampled_mean**2/2,resampled_mean**3/6,resampled_mean**4/24])
print('p-value of the chi-squared G.O.F. test for a resampled version of \
Bortkiewicz\'s data having Poisson distribution: '+str(ss.chisquare(resampled_horse_kicks,resampled_expected)[1]))

print('Poisson weight of the resampled distribution: '+str(pois.poisson_weight(resampled_P)[0]))

*When the same source is observed through a larger sample our beliefs about its Poisson-ness change dramatically if we use hypothesis tests, but not if we estimate the latent weight of Poisson models!*

# Coding section 6
## Estimating the exchangeable weight --- how to determine an adequate sample size?

Because it is unlikely that a read will contain, say, $20$ CpGs, we focus on \"triplets,\" sets of $3$ consecutive CpGs whose possible methylation configurations are $(0,0,0),(0,0,1),(0,1,0),(0,1,1),(1,0,0),(1,0,1),(1,1,0),$ and $(1,1,1)$. The configurations of many triplets are fully observed within dozens of reads in a WGBS experiment of typical coverage. 

In determining what sample size should be used, we want to find a source that is particularly pathological. That is, a source $P$ for which $\lambda(\hat P_n)$ has large negative bias and large standard error. Then if a sample size $n$ gives acceptably small bias and standard error in the \"worst-case\" source, estimates of exchangeable weight should be acceptably accurate for any source.

We notice that estimating the exchangeable weight of source which is itself exchangeable tends to give the largest negative bias and standard error. Therefore we place a grid over $\mathcal Q$, the exchangeable distributions, and test biases and standard errors for various sample sizes.

In [ ]:
np.random.seed(999)

def exchangeable_weight(P):#Here, P is a source on {0,1}^3
    return P[0]+3*min([P[1],P[2],P[4]])+3*min([P[3],P[5],P[6]])+P[7]

E0 = np.array([1,0,0,0,0,0,0,0])
E1 = np.array([0,1/3,1/3,0,1/3,0,0,0])
E2 = np.array([0,0,0,1/3,0,1/3,1/3,0])
E3 = np.array([0,0,0,0,0,0,0,1])

import simplex_grid as sg
mix_weights = sg.simplex_grid(4,20)/20.

In [ ]:
test_sources = []
for weight in mix_weights:
    test_sources.append(weight[0]*E0+weight[1]*E1+weight[2]*E2+weight[3]*E3)
    
#Is n=30 an adequate sample size?

bias = []
standard_error = []
estimates = []
for P in test_sources:
    estimate = []
    for k in range(100):
        Phat = np.random.multinomial(30,P)/30.
        estimate.append(exchangeable_weight(Phat))
    bias.append(np.mean(estimate)-1)
    standard_error.append(np.std(estimate))
    estimates.append(estimate)

#Let's look at the worst case bias
print('Source with worst-case bias: '+str(test_sources[np.argmin(bias)]))
plt.hist(estimates[np.argmin(bias)],bins=6,density=True)
plt.axvline(1,color='red',linestyle='dashed')
plt.xlim(0,1.1)
plt.xlabel(r'$\lambda_{Q}(\hat P_{30})$')
plt.title(r'Worst-case bias estimates, $n=30$')

In [ ]:
#Let's look at the worst case standard_error
print('Source with worst-case standard error: '+str(test_sources[np.argmax(standard_error)]))
plt.hist(estimates[np.argmax(standard_error)],density=True,bins=6)
plt.axvline(1,color='red',linestyle='dashed')
plt.xlim(0,1.1)
plt.xlabel(r'$\lambda_{Q}(\hat P_{30})$')
plt.title(r'Worst-case standard error estimates, $n=30$')

Clearly $n=30$ is not enough samples. How about $n=100$?

In [ ]:
bias = []
standard_error = []
estimates = []
for P in test_sources:
    estimate = []
    for k in range(100):
        Phat = np.random.multinomial(100,P)/100.
        estimate.append(exchangeable_weight(Phat))
    bias.append(np.mean(estimate)-1)
    standard_error.append(np.std(estimate))
    estimates.append(estimate)

In [ ]:
print('Source with worst-case bias: '+str(test_sources[np.argmin(bias)]))
plt.hist(estimates[np.argmin(bias)],bins=6,density=True)
plt.axvline(1,color='red',linestyle='dashed')
plt.xlim(0,1.1)
plt.xlabel(r'$\lambda_{Q}(\hat P_{100})$')
plt.title(r'Worst-case bias estimates, $n=100$')

In [ ]:
print('Source with worst-case standard error: '+str(test_sources[np.argmax(standard_error)]))
plt.hist(estimates[np.argmax(standard_error)],density=True,bins=6)
plt.axvline(1,color='red',linestyle='dashed')
plt.xlim(0,1.1)
plt.xlabel(r'$\lambda_{Q}(\hat P_{100})$')
plt.title(r'Worst-case standard error estimates, $n=100$')

$n=100$ seems like a better sample size but there are still several estimates close to 0.8. We want the estimates to be near $1$ with high probability. How about correcting the bias by bootstrap resampling? Let's try, using the worst-case-bias test source.

In [ ]:
np.random.seed(123)
P = test_sources[np.argmin(bias)]
BS_corrected_estimates = []
Lambdas = []
BS_lambdas = []
for k in range(1000): #Take 1000 samples of size n=100 from the pathological source P
    Phat = np.random.multinomial(100,P)/100.
    Lambda = exchangeable_weight(Phat) #Compute the exchangeable weight from the empirical measure
    Lambdas.append(Lambda)
    BS_estimates = []
    for j in range(1000):
        Phat_star = np.random.multinomial(100,Phat)/100. #Resample the empirical measure, taking 100 observations
        BS_estimates.append(exchangeable_weight(Phat_star)) #Compute the exchangeable weight of the 
                                                            #empirical measure associated with this resample
    BS_lambdas.append(BS_estimates)

corrected_lambdas = []
for j in range(100):
    corrected_lambdas.append(2*Lambdas[j]-np.mean(BS_lambdas[j]))
plt.hist(Lambdas,density=True,bins=6,alpha=0.5,color='red',label=r'Uncorrected $\hat\lambda_{100}$')
plt.hist(corrected_lambdas,density=True,bins=6,alpha=0.5,color='green',label=r'BS-corrected $\hat\lambda_{100}$')
plt.legend(loc='upper left')
plt.axvline(1,color='red',linestyle='dashed')
plt.xlim(0,1.1)

How well can we estimate the exchangeable weight on a more typical (less pathological source)? Let's pick up a source uniformly at random from the set of distributions on a triplet's methylation configurations.

In [ ]:
P = np.random.dirichlet([1]*8)#Choosing a source uniformly at random from P({0,1}^3)
Lambda = exchangeable_weight(P)
Lambda_hats = []
for j in range(1000):
    Phat = np.random.multinomial(100,P)/100.
    Lambda_hat = exchangeable_weight(Phat)
    resampled_lambdas = []
    for j in range(1000):
        Phat_star = np.random.multinomial(100,Phat)/100.
        Lambda_star = exchangeable_weight(Phat_star)
        resampled_lambdas.append(Lambda_star)
    Lambda_hats.append(2*Lambda_hat-np.mean(resampled_lambdas))
plt.xlim(0,1)
plt.hist(Lambda_hats,density=True,bins=20)
plt.axvline(Lambda,color='red',linestyle='dashed')
plt.xlabel(r'$\lambda_{Q}(\hat P_100)$, corrected by bootstrap bias')
plt.title('Estimating the exchangeable weight of a random source')

It seems like $n=100$ may not be such a bad sample size for estimating the exchangeable weight of most sources.